In [1]:
from anti_qsweepy.drivers import *
from anti_qsweepy.routines import data_mgmt
from anti_qsweepy.routines.helper_functions import *
import anti_qsweepy.routines.differential_evolution as di
import anti_qsweepy.routines.jpa_tuning as jt
from numpy import *
import scipy.optimize as so
import time
import imp

In [80]:
imp.reload(di)
imp.reload(jt)

<module 'anti_qsweepy.routines.jpa_tuning' from 'C:\\simple_lab_scripts\\anti_qsweepy\\routines\\jpa_tuning.py'>

In [2]:
na = Agilent_PNA.NetworkAnalyzer('PNA-X')
#na = RS_ZNB20.NetworkAnalyzer('ZNB20')
#bias_source = Keithley_2400.CurrentSource('2400')
#bias_source = Keithley_6221.CurrentSource('GPIB0::10::INSTR')
#bias_source = Keithley_2651.MagnetSupply("NNDAC", rate = 1.)
pump = Agilent_PSG.Generator('TCPIP0::10.1.0.79::inst0::INSTR')

In [4]:
pump.power(0.55)

0.55

In [13]:
pump.output('on')

'1'

In [7]:
pump.freq(14.3e9)

14300000000.0

In [80]:
target_gain  = 22
#bias_range = (2.5e-3, 4.55e-3)
bias_range = (3e-3, 4.5e-3)
bias_source_range = 10e-3
bias_source_limit = 10
Fcent = 6.8e9
f_cent_range = (6e9,7.2e9)
Fspan = 0.8e9
Npoints = 350
power = -30
bw = 10e3
pump_range = (-7,10)
#pump_range = (-2,18)

In [81]:
na.num_of_points(Npoints)
na.freq_center_span((Fcent,Fspan))
na.bandwidth(bw)
na.power(power)

na.sweep_type("LIN")

bias_source.range(bias_source_range)
bias_source.limit(bias_source_limit)

pump.freq(Fcent*2.)

Fna = na.freq_points()

na.output('on')
bias_source.setpoint(0.)
bias_source.output('on')

1

In [88]:
#Reference
pump.output('off')
na.soft_trig_arm()
ref = na.read_data()
db_ref = mean(20*log10(abs(ref)))
print("Reference level: {:f}db".format(db_ref))
pump.output('on')
na.soft_trig_abort()

Reference level: -42.086122db


In [89]:
class FuncMin():
    """
    def func_min(self,x):
        #x[0] - bias
        #x[1] - pump
        pump.power(x[1])
        bias_source.setpoint(x[0])
        #time.sleep(0.01)
        gain = 20*log10(abs(na.read_data()))-db_ref
        diff = gain-target_gain
        return mean(diff**2) + 0.5*diff[int(len(diff)/2)]**2
    """
    def func_min(self,x):
        #x[0] - bias
        #x[1] - pump
        pump.power(x[1])
        bias_source.setpoint(x[0])
        #time.sleep(0.01)
        #gain = 20*log10(abs(na.read_data()))-db_ref
        #diff = gain-target_gain
        
        gain = abs(na.read_data()/ref)
        diff = gain-10**(target_gain/20)
        
        f_cent,span = na.freq_center_span()
        na.sweep_type('cw')
        na.freq_cw( f_cent + 2e6 )
        
        data_on = na.read_data()
        pump.output('off')
        data_off = na.read_data()
        g = abs(data_on/data_off)
        sigma = std(g)
        snr_on = mean(abs(data_on))/std(abs(data_on))
        snr_off = mean(abs(data_off))/std(abs(data_off))
        snr_gain = snr_on/snr_off
        
        pump.output('on')
        na.sweep_type('lin')
        
        return mean(diff**2) + 0*0.5*diff[int(len(diff)/2)]**2 - snr_gain**2

    def func_min_vect(self,x):
        if len(shape(x)) == 2:
            res = zeros(shape(x)[0])
            for i,val in enumerate(x):
                res[i] = self.func_min(val)
            return res
        elif len(shape(x)) == 1:
            return self.func_min(x)
        else:
            raise Ecxeption('Invalid argument shape')

In [75]:
na.soft_trig_arm()
FuncMin().func_min_vect([[3.123e-3,0],])
na.soft_trig_abort()

NameError: name 'snr' is not defined

## Noisy

In [ ]:
class FuncMin():
    
    def ref(self):
        pump.output('off')
        na.soft_trig_arm()
        self.ref = na.read_data()
        print("Reference level: {:f}db".format(20*log10(abs(db_ref))))
        pump.output('on')
        na.soft_trig_abort()
    
    def func_min(self,x):
        #x[0] - bias
        #x[1] - pump
        pump.power(x[1])
        bias_source.setpoint(x[0])
        #time.sleep(0.01)
        gain = 20*log10(abs(na.read_data()))-db_ref
        diff = gain-target_gain
        
        f_cent,span = na.freq_center_span()
        na.sweep_type('cw')
        na.freq_cw( f_cent + 2e6 )
        S21 = 20*log10(abs(na.read_data()))
        sigma = std(S21)
        na.sweep_type('lin')
        return 0*mean(diff**2) + 0*0.5*diff[int(len(diff)/2)]**2 + (10*sigma)**2

    def func_min_vect(self,x):
        if len(shape(x)) == 2:
            res = zeros(shape(x)[0])
            for i,val in enumerate(x):
                res[i] = self.func_min(val)
            return res
        elif len(shape(x)) == 1:
            return self.func_min(x)
        else:
            raise Ecxeption('Invalid argument shape')

In [218]:
na.freq_cw( 6e9 + 2e6 )

6002000000.0

In [5]:
a =zeros((3,2))

In [8]:
std(a, axis = 1)

array([0., 0., 0.])

In [195]:
class FuncMin():
    def func_min(self,x):
        #x[0] - bias
        #x[1] - pump
        #x[2] - frequency
        pump.power(x[1])
        pump.freq(x[2]*2)
        bias_source.setpoint(x[0])
        na.freq_center_span((x[2],Fspan))
        #time.sleep(0.01)
        gain = 20*log10(abs(na.read_data()))-db_ref
        diff = gain-target_gain
        return mean(diff**2) + 0.5*diff[int(len(diff)/2)]**2

    def func_min_vect(self,x):
        if len(shape(x)) == 2:
            res = zeros(shape(x)[0])
            for i,val in enumerate(x):
                res[i] = self.func_min(val)
            return res
        elif len(shape(x)) == 1:
            return self.func_min(x)
        else:
            raise Ecxeption('Invalid argument shape')

In [43]:
#x[0] - bias
#x[1] - pump
#x[2] - freq
def func_min1(x):
    pump.power(x[1])
    bias_source.setpoint(x[0])
    na.freq_center_span((x[2], Fspan))
    pump.freq(2*x[2])
    time.sleep(0.01)
    gain = 20*log10(abs(na.read_data()))-db_ref
    diff = gain-target_gain
    return mean(diff**2) + 0.0*diff[int(len(diff)/2)]**2 

In [ ]:
x0 = array([ mean(bias_range)*1e3 , mean(pump_range) ])
so.basinhopping(func_min, x0, niter=10)

In [ ]:
so.shgo(func_min, [bias_range, pump_range])

In [94]:
n_init = 20
init_pop = zeros((n_init**2,2))
i=0
for val0 in linspace(bias_range[0], bias_range[1],n_init):
    for val1 in linspace(pump_range[0], pump_range[1],n_init):
        init_pop[i] = [val0,val1]
        i+=1

In [24]:
t1=time.time()
na.soft_trig_arm()
res = so.differential_evolution(func_min,  [bias_range, pump_range], tol = 0.06, popsize = 100, maxiter = 30)
na.soft_trig_abort()
time.time()-t1

485.64856791496277

In [44]:
t1=time.time()
na.soft_trig_arm()
res = so.differential_evolution(func_min1,  [bias_range, pump_range, (Fcent-0.01e9, Fcent+0.01e9)], tol = 0.06, 
                                popsize = 50, maxiter = 30)
na.soft_trig_abort()
time.time()-t1

692.4740211963654

In [153]:
res = so.differential_evolution(func_min1,  [bias_range, pump_range], tol = 0.06)

In [31]:
res

     fun: 8.283114394223693
 message: 'Optimization terminated successfully.'
    nfev: 678
     nit: 6
 success: True
       x: array([ 3.67744812e-03, -6.07546917e+00])

In [177]:
res['x']

array([ 0.00330865, -1.36071587])

## Modified differential evolution

In [176]:
t1=time.time()
na.soft_trig_arm()
solver = di.DifferentialEvolutionSolver(FuncMin().func_min_vect,  
                                [bias_range, pump_range], tol = 0.06, 
                                popsize = 50, minpopsize = 5, maxiter = 30 )
res = solver.solve()
na.soft_trig_abort()
print(time.time()-t1) 

63.33210492134094


In [90]:
t1=time.time()
na.soft_trig_arm()
solver = di.DifferentialEvolutionSolver(FuncMin().func_min_vect,  
                                [bias_range, pump_range], tol = 0.06, 
                                popsize = 50, minpopsize = 5, maxiter = 30 )
res = solver.solve()
na.soft_trig_abort()
print(time.time()-t1) 

235.9547517299652


In [196]:
t1=time.time()
na.soft_trig_arm()
solver = di.DifferentialEvolutionSolver(FuncMin().func_min_vect,  
                                [bias_range, pump_range, f_cent_range], tol = 0.06, 
                                popsize = 50, minpopsize = 5, maxiter = 50 )
res = solver.solve()
na.soft_trig_abort()
print(time.time()-t1) 

305.62023425102234


In [168]:
pump.output()

'1\n'

In [17]:
bias_source.setpoint()

0.00353

In [169]:
bias_source.output()

1

In [13]:
na.soft_trig_abort()

In [37]:

bias_source.setpoint(3.9304341e-3)
pump.power(-2.98789442)

-2.98789442

In [52]:
shape(array([[1,2],[1,2],[1,2]]))

(3, 2)

In [48]:
def test_func(a,b):
    return a**b
args = (2,3)
test_func(*args)

8

In [50]:
random.RandomState(1)

In [128]:
x = array([[1,2,3],[1,2,3],[4,5,6],[8,9,7]])

In [133]:
delete(x, [2,1],axis = 0)

array([[1, 2, 3],
       [8, 9, 7]])

In [122]:
a = array([2,3,56,2,1])

In [123]:
argsort(a)


array([4, 0, 3, 1, 2], dtype=int64)

In [125]:
delete(a, [2,3])

array([2, 3, 1])

In [198]:
import os

In [201]:
os.path.split('abc\\def')

('abc', 'def')

In [91]:
855-233

622

In [87]:
l =[1,2,3]

In [88]:
l=[]
l+=[1,]

In [89]:
l

[1]

In [11]:
std([1+1j,2+0.5j,3+0.3j])

0.8679477710861025

In [13]:
(1+1.j).real()

TypeError: 'float' object is not callable